In [ ]:
!pip install geemap
!pip install -U google-api-python-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 94.7 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.160.0
    Uninstalling google-api-python-client-2.160.0:
      Successfully uninstalled google-api-python-client-2.160.0


In [ ]:
import ee
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-mwu336')

In [24]:
# Load the MODIS MCD12Q2 dataset and filter by year
modis = (ee.ImageCollection("MODIS/061/MCD12Q2")
         .filterDate('2023-01-01', '2023-12-31')  # Select year
         .mean())

# Load states boundary
states = ee.FeatureCollection("TIGER/2018/States")

# List of state FIPS codes
stateFIPS = [
    '08', '17', '18', '19', '20', '21', '26', '27', '29', '31',
    '37', '38', '39', '42', '46', '47', '48', '55'
]

# Filter states based on the FIPS codes
selected_states = states.filter(ee.Filter.inList('STATEFP', stateFIPS))

# Clip MODIS image to the selected states
modis_states = modis.clip(selected_states)

# Reduce the image to extract data at pixel level for each state
modis_reduced = modis_states.reduceRegions(
    collection=selected_states,
    reducer=ee.Reducer.mean(),
    scale=500  # MODIS resolution is 500m
)

# List of columns to convert from Unix Epoch Days to DOY
columns_to_convert = [
    'Dormancy_1', 'Dormancy_2', 'EVI_Amplitude_1', 'EVI_Amplitude_2', 'EVI_Area_1', 'EVI_Area_2',
    'EVI_Minimum_1', 'EVI_Minimum_2', 'Greenup_1', 'Greenup_2', 'Maturity_1', 'Maturity_2',
    'MidGreendown_1', 'MidGreendown_2', 'MidGreenup_1', 'MidGreenup_2', 'NumCycles', 'Peak_1', 'Peak_2',
    'QA_Detailed_1', 'QA_Detailed_2', 'QA_Overall_1', 'QA_Overall_2', 'Senescence_1', 'Senescence_2'
]

# Function to convert Unix Epoch Days to DOY
def convert_specified_columns_to_doy(feature):
    updates = {}
    for col in columns_to_convert:
        value = feature.getNumber(col)  # Get column value
        doy = ee.Date(value.multiply(86400000)).getRelative('day', 'year')  # Convert to DOY
        updates[col] = doy
    return feature.set(updates)

# Apply conversion function
feature_collection = modis_reduced.map(convert_specified_columns_to_doy)

# Print the feature collection
# print(feature_collection.getInfo())

# Export the data to Google Drive as CSV
task = ee.batch.Export.table.toDrive(
    collection=feature_collection,
    description='MODIS_States_2023',
    folder='MODIS',
    fileFormat='CSV'
)

task.start()
print("Export started...")

Export started...
